In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import norm
from scipy.interpolate import CubicSpline
# from scipy.optimize import curve_fit
# from scipy import stats
# %matplotlib inline

In [ ]:
def displacements(frame0, frame_zv):
    
    dx = []
    dy = []
    dr = []
#     dx2 = []
    
    for part in frame0['particle']:
        
        if (part in frame_zv['particle'].unique()):
            mask0 = frame0['particle'] == part
            x0 = frame0[mask0]['x'].item()
            y0 = frame0[mask0]['y'].item()

            mask_zv = frame_zv['particle'] == part
            x_zv = frame_zv[mask_zv]['x'].item()
            y_zv = frame_zv[mask_zv]['y'].item()

            
            dx_ = x_zv - x0
            dy_ = y_zv - y0
            dx.append( dx_ )
            dy.append( dy_ )
            
            if dx_ < 0:
                dr_ = - np.sqrt(dx_**2 + dy_**2)
            else:
                dr_ = np.sqrt(dx_**2 + dy_**2)
            
            dr.append( dr_ )

#             dx2.append( dx**2 ) 
            
#     dx2_mean = sum(dx2) / len(dx2)
    
    
    return dx, dy, dr#, dx2_mean
    

In [ ]:
# Gaussian approximation of MSD

def Gauss_MSD(dx):
    
#     sigma = np.mean( [x*x for x in dx] )
    sigma_list = [x*x for x in dx]
    sigma = np.sum(sigma_list) / len(sigma_list)
    
    G0 = np.zeros_like(dx)
    
    factor1 = np.sqrt(1 / (2 * np.pi * sigma)) #1 / (np.pi * sigma) #
    factor2 = 2 * sigma #sigma #
    
    for i, dx_ in enumerate(dx):

        G0[i] = factor1 * np.exp( -dx_**2 / factor2 )
    
    
    return G0

In [ ]:
file_name_prefix = '3_0_2000' #'2_4000_6000' #

In [ ]:
def read_data():
    
    file_path = f'data_out/backup_{file_name_prefix}/S6T34to18_t_corrected_{file_name_prefix}.csv'
    
    usecols = ['y', 'x', 'frame', 'particle']
    
    data = pd.read_csv(file_path, usecols=usecols)
    
    
    return data

In [ ]:
%%time


data = read_data()
# print(data.head())
data_frame0 = data[data['frame'] == 0]
max_frame = data['frame'].values[-1] + 1
frame_zv = 450 # t*



f0 = data[data['frame'] == 0]
f_zv = data[data['frame'] == frame_zv]


dx, dy, dr0 = displacements(f0, f_zv)

dr_cut = list(filter(lambda x: abs(x) < 10 , dr0))

a1 = [-2] #* 5[0] * 3 #
a2 = [6] #[0.5] * 3
# a3 = [0.25] * 5
# a4 = [-0.5] * 5
# a5 = [-1.5] * 3
# a6 = [1] * 3


dr = dr_cut #+ a1 + a2 # + a3 + a4 + a5 + a6 #+ a1 + a2# + [1,1,1,1,1,1,1,1] #[0, 0.01, 0.1, 1, 1.5, 2]
# normal_distr.insert(-1, 0.2)


# P, bin_edges = np.histogram(dr0, bins=50, density=True)
# dr = 0.5 * (bin_edges[1:] + bin_edges[:-1])


normal_distr = list(Gauss_MSD(dr))


# dr = np.arange(-10, 10, 0.0001)
# dr = r[1] - r[0]


In [ ]:
dr, normal_distr = zip(*sorted(zip(dr, normal_distr)))
print(len(dr))

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(figsize=(10, 6))

plt.rcParams['axes.labelsize'] = 18

# plt.hist(dr, bins=15, density=True, alpha=0.6, color='g')

plt.plot(dr, normal_distr, ls='-', lw=2, c='k') #'mediumpurple'
sns.kdeplot(data=dr, bw_adjust=1.8, ls='--', lw=1, marker='.', ms=20, markevery=4, markerfacecolor="none", c='k')#, cut=0) #1.7
#1.8


# plt.ylim([0.007, 0.3])
plt.xlim([-11, 11])
plt.ylim([0.007, 0.3])

px_to_micron = 0.06905
xticks = [-0.6/px_to_micron, -0.3/px_to_micron, 0, 0.3/px_to_micron, 0.6/px_to_micron] #[-10, -5, 0, 5, 10]

xticks_micron = [i * px_to_micron for i in xticks]

plt.xticks(xticks, xticks_micron, fontsize=20)
plt.yticks(fontsize=20)

plt.xlabel(r'r $(\mu m)$', fontsize=20)
plt.ylabel(r'$G_s$', fontsize=20)


# ax.set(ylabel=r'$G_s$',
#       xlabel=r'r $(\mu m)$')

plt.yscale('log')

# plt.title('cool3, 0-2000 frames = [4.4, 5] min')

# plt.legend (fontsize=14)

In [ ]:
# # save image

# # plt.savefig(f'../results/T34to18/Gs_{file_name_prefix}.png')
# plt.savefig(f'/Users/miroshni/Documents/Unifr/Unifr_work/glass_transition_in_emulsions/TrackPy/microgels/feb20/our_article/images/Gs_{file_name_prefix}.png', format='png')
# plt.savefig(f'/Users/miroshni/Documents/Unifr/Unifr_work/glass_transition_in_emulsions/TrackPy/microgels/feb20/our_article/images/Gs_{file_name_prefix}.svg', format='svg')

In [ ]:
# # write to file
# DIR = f'data_out/backup_{file_name_prefix}/'

# file_Distr = DIR + f'Distribution_function_{file_name_prefix}.csv'
# file_Normal = DIR + f'Normal_Distribution_function_{file_name_prefix}.csv'

# # get density from seaborn
# x_distr, y_distr = sns.kdeplot(dr).lines[0].get_data()

# data_out_dict1 = {"r": x_distr, "G": y_distr} # px in pixels
# data_out_dict2 = {"r": dr, "G": normal_distr} # px in pixels

# data_out1 = pd.DataFrame(data_out_dict1)
# data_out2 = pd.DataFrame(data_out_dict2)

# data_out1.to_csv(file_Distr, index=False)
# data_out2.to_csv(file_Normal, index=False)